In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import shutil
from datetime import datetime, timedelta
import pandas as pdX
import numpy as np

/Users/ijongseung/market_data/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [82]:
def download_and_process_kpx_data(chromedriver_path, chrome_binary_path, download_dir):
    # === 다운로드 경로 설정 ===
    os.makedirs(download_dir, exist_ok=True)

    # === Selenium 옵션 ===
    options = Options()
    options.binary_location = chrome_binary_path
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)

    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)

    # === 타겟 URL 접근 ===
    target_url = "https://epsis.kpx.or.kr/epsisnew/selectEkmaFucUpfChart.do?menuId=040100"
    driver.get(target_url)
    time.sleep(3)

    # === 엑셀 다운로드 ===
    driver.execute_script("excelExport();")
    print("엑셀 다운로드 실행됨!")
    time.sleep(5)  # 다운로드 대기

    # === 다운로드된 파일 확인 ===
    downloaded_files = os.listdir(download_dir)
    print("다운로드 완료된 파일 목록:", downloaded_files)
    driver.quit()

    # === 가장 최근 파일 찾기 ===
    valid_files = [f for f in downloaded_files if f.endswith('.xlsx') and not f.startswith('~$')]
    latest_file = max([os.path.join(download_dir, f) for f in valid_files], key=os.path.getctime)
    print(f"가장 최근 파일: {latest_file}")

    # === header=2로 3번째 행부터 컬럼 읽기 ===
    df = pd.read_excel(latest_file, header=2, engine='openpyxl')
    print("정확히 읽힌 데이터:")
    print(df.head())

    # === 필요한 열 선택 (기간 + 마지막 5개 열) ===
    df_filtered = df.iloc[:, [0, -5, -4, -3, -2, -1]]

    # === 숫자형 데이터 정제 ===
    for col in df_filtered.columns[1:]:
        df_filtered[col] = (
            df_filtered[col]
            .astype(str)
            .str.replace(',', '', regex=False)
            .str.replace('#', '', regex=False)
            .str.strip()
        )
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

    df_filtered = df_filtered.fillna(0)

    # === 컬럼명 정제 ===
    df_filtered.rename(columns={
        df_filtered.columns[0]: '기간',
        df_filtered.columns[1]: '원자력',
        df_filtered.columns[2]: '유연탄',
        df_filtered.columns[3]: '무연탄',
        df_filtered.columns[4]: '유류',
        df_filtered.columns[5]: 'LNG'
    }, inplace=True)

    # === 저장 ===
    output_path = os.path.join(download_dir, "fuel_unit_cost_only.xlsx")
    df_filtered.to_excel(output_path, index=False)
    print(f"\n정제된 연료비단가 데이터 저장 완료: {output_path}")

    return output_path, df_filtered

In [83]:
chromedriver_path = "/Users/ijongseung/market_data/chromedriver-mac-arm64/chromedriver"
chrome_binary_path = "/Users/ijongseung/Downloads/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
download_dir = "/Users/ijongseung/Downloads/kpx_download"

output_path, processed_df = download_and_process_kpx_data(chromedriver_path, chrome_binary_path, download_dir)

print(f"최종 저장 위치: {output_path}")
print(processed_df)

KeyboardInterrupt: 

## 전력수급 
 - 5분단위 데이터 크롤링하기

In [ ]:
# 경로 설정
chromedriver_path = "/Users/ijongseung/market_data/chromedriver-mac-arm64/chromedriver"
chrome_binary_path = "/Users/ijongseung/Downloads/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
download_dir = "/Users/ijongseung/Downloads/kpx_download"
os.makedirs(download_dir, exist_ok=True)

# Selenium 옵션
options = Options()
options.binary_location = chrome_binary_path
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # 대상 URL 접속
    target_url = "https://epsis.kpx.or.kr/epsisnew/selectEkgeEpsMepRealChart.do?menuId=030300"
    driver.get(target_url)
    wait = WebDriverWait(driver, 20)

    # 수집할 날짜 범위 설정
    start_date = datetime(2025, 1, 1)
    end_date = datetime(2025, 1, 5)
    delta = timedelta(days=1)

    current_date = start_date
    while current_date <= end_date:
        year = current_date.year
        month = current_date.month
        day = current_date.day
        formatted_date = f"{year}-{month:02d}-{day:02d}"
        print(f"\n{formatted_date} 작업 시작")

        # DatePicker input에 날짜 입력
        datepicker_input = wait.until(EC.element_to_be_clickable((By.ID, 'selEndDate')))
        driver.execute_script(f"arguments[0].value = '{formatted_date}';", datepicker_input)
        print(f"{formatted_date} 날짜 입력 완료")
        time.sleep(1)

        # CSV 다운로드 버튼 클릭
        csv_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@onclick, 'csvExport')]")))
        csv_button.click()
        print("CSV 다운로드 클릭 완료")

        # 다운로드 대기
        time.sleep(7)

        # 다운로드된 CSV 파일명 변경
        files = os.listdir(download_dir)
        files = [f for f in files if f.endswith('.csv')]
        if files:
            latest_file = max([os.path.join(download_dir, f) for f in files], key=os.path.getctime)
            new_name = f"kpx_{year}_{month:02d}_{day:02d}.csv"
            shutil.move(latest_file, os.path.join(download_dir, new_name))
            print(f"{new_name} 저장 완료")
        else:
            print(f"{formatted_date} CSV 파일 다운로드 실패")

        # 다음 날짜로 이동
        current_date += delta

    print("\n데이터 수집 완료!")

finally:
    driver.quit()
    print("브라우저 종료 완료!")

# === CSV 병합 ===
print("\nCSV 병합 시작...")

csv_files = [os.path.join(download_dir, f) for f in os.listdir(download_dir) if f.endswith('.csv')]
df_list = []

for file in sorted(csv_files):
    df = pd.read_csv(file, encoding='cp949')
    date_str = file.split('kpx_')[-1].replace('.csv', '')
    df['날짜'] = date_str
    df_list.append(df)

merged_df = pd.concat(df_list, ignore_index=True)

# 병합 결과 저장
output_file = "/Users/ijongseung/Downloads/kpx_merged_2025_01_01_to_2025_01_20.csv"
merged_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"병합된 CSV 저장 완료: {output_file}")


2025-01-01 작업 시작
2025-01-01 날짜 입력 완료
CSV 다운로드 클릭 완료
kpx_2025_01_01.csv 저장 완료

2025-01-02 작업 시작
2025-01-02 날짜 입력 완료
CSV 다운로드 클릭 완료
kpx_2025_01_02.csv 저장 완료

2025-01-03 작업 시작
2025-01-03 날짜 입력 완료
CSV 다운로드 클릭 완료
kpx_2025_01_03.csv 저장 완료

2025-01-04 작업 시작
2025-01-04 날짜 입력 완료
CSV 다운로드 클릭 완료
kpx_2025_01_04.csv 저장 완료

2025-01-05 작업 시작
2025-01-05 날짜 입력 완료
CSV 다운로드 클릭 완료
kpx_2025_01_05.csv 저장 완료

데이터 수집 완료!
브라우저 종료 완료!

CSV 병합 시작...
병합된 CSV 저장 완료: /Users/ijongseung/Downloads/kpx_merged_2025_01_01_to_2025_01_20.csv


In [89]:
# 데이터 호출
df_demand = pd.read_csv("/Users/ijongseung/Downloads/kpx_merged_2025_01_01_to_2025_02_20.csv")
df_demand["일시"] = pd.to_datetime(df_demand["일시"])

df_demand

,일시,공급능력(MW),현재부하(MW),공급예비력(MW),공급예비율(%),기온(°C),날짜
0,2025-03-24 17:35:00,87264,65708,21557,32.8,"""""",2025_01_01
1,2025-03-24 17:30:00,87356,65467,21888,33.4,"""""",2025_01_01
2,2025-03-24 17:25:00,87449,65299,22150,33.9,"""""",2025_01_01
3,2025-03-24 17:20:00,87560,65324,22237,34.0,"""""",2025_01_01
4,2025-03-24 17:15:00,87600,65212,22388,34.3,"""""",2025_01_01
...,...,...,...,...,...,...,...
2115,2025-03-24 00:20:00,84581,52322,32259,61.7,"""""",2025_01_10
2116,2025-03-24 00:15:00,84568,52874,31694,59.9,"""""",2025_01_10
2117,2025-03-24 00:10:00,82042,53307,28735,53.9,"""""",2025_01_10
2118,2025-03-24 00:05:00,82026,53453,28573,53.5,"""""",2025_01_10


In [121]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import shutil
from datetime import datetime, timedelta
import pandas as pd

# 경로 설정
chromedriver_path = "/Users/ijongseung/market_data/chromedriver-mac-arm64/chromedriver"
chrome_binary_path = "/Users/ijongseung/Downloads/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
download_dir = "/Users/ijongseung/Downloads/kpx_download"
os.makedirs(download_dir, exist_ok=True)

# Selenium 옵션
options = Options()
options.binary_location = chrome_binary_path
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # 대상 URL 접속
    target_url = "https://epsis.kpx.or.kr/epsisnew/selectEkgeEpsMepRealChart.do?menuId=030300"
    driver.get(target_url)
    wait = WebDriverWait(driver, 20)

    # 수집할 날짜 범위 설정
    start_date = datetime(2025, 1, 1)
    end_date = datetime(2025, 1, 15)
    delta = timedelta(days=1)

    current_date = start_date
    while current_date <= end_date:
        year = current_date.year
        month = current_date.month
        day = current_date.day
        formatted_date = f"{year}-{month:02d}-{day:02d}"
        print(f"\n{formatted_date} 작업 시작")

        # DatePicker input에 날짜 입력 + change 이벤트 발생
        datepicker_input = wait.until(EC.element_to_be_clickable((By.ID, 'selEndDate')))
        driver.execute_script(
            f"arguments[0].value = '{formatted_date}'; arguments[0].dispatchEvent(new Event('change'));", 
            datepicker_input
        )
        print(f"{formatted_date} 날짜 입력 및 이벤트 발생 완료")
        time.sleep(3)  # 데이터 갱신 대기

        # CSV 다운로드 버튼 클릭
        csv_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@onclick, 'csvExport')]")))
        csv_button.click()
        print("CSV 다운로드 클릭 완료")

        # 다운로드 대기
        time.sleep(7)

        # 다운로드된 CSV 파일명 변경
        files = os.listdir(download_dir)
        files = [f for f in files if f.endswith('.csv')]
        if files:
            latest_file = max([os.path.join(download_dir, f) for f in files], key=os.path.getctime)
            new_name = f"kpx_{year}_{month:02d}_{day:02d}.csv"
            shutil.move(latest_file, os.path.join(download_dir, new_name))
            print(f"{new_name} 저장 완료")
        else:
            print(f"{formatted_date} CSV 파일 다운로드 실패")

        # 다음 날짜로 이동
        current_date += delta

    print("\n데이터 수집 완료!")

finally:
    driver.quit()
    print("브라우저 종료 완료!")

# === CSV 병합 ===
print("\nCSV 병합 시작...")

csv_files = [os.path.join(download_dir, f) for f in os.listdir(download_dir) if f.endswith('.csv')]
df_list = []

for file in sorted(csv_files):
    df = pd.read_csv(file, encoding='cp949')
    date_str = file.split('kpx_')[-1].replace('.csv', '')
    df['날짜'] = date_str
    df_list.append(df)

merged_df = pd.concat(df_list, ignore_index=True)

# 병합 결과 저장
output_file = "/Users/ijongseung/Downloads/kpx_merged_2025_01_01_to_2025_01_05.csv"
merged_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"병합된 CSV 저장 완료: {output_file}")



2025-01-01 작업 시작
2025-01-01 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_01.csv 저장 완료

2025-01-02 작업 시작
2025-01-02 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_02.csv 저장 완료

2025-01-03 작업 시작
2025-01-03 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_03.csv 저장 완료

2025-01-04 작업 시작
2025-01-04 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_04.csv 저장 완료

2025-01-05 작업 시작
2025-01-05 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_05.csv 저장 완료

2025-01-06 작업 시작
2025-01-06 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_06.csv 저장 완료

2025-01-07 작업 시작
2025-01-07 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_07.csv 저장 완료

2025-01-08 작업 시작
2025-01-08 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_08.csv 저장 완료

2025-01-09 작업 시작
2025-01-09 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_09.csv 저장 완료

2025-01-10 작업 시작
2025-01-10 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_10.csv 저장 완료

2025-01-11 작업 시작
2025-01-11 날짜 입력 및 이벤트 발생 완료
CSV 다운로드 클릭 완료
kpx_2025_01_11.csv 저장 완료

2025-01-12 작업 시작
2025-01-12 날짜 입력 및 이벤트 발생

In [122]:
merged_df["일시"] = pd.to_datetime(merged_df["일시"])
merged_df.set_index("일시", inplace=True)


In [123]:
merged_df = merged_df.iloc[:, 0:3]

In [135]:
merged_df.sort_index(inplace=True, ascending=True)
